In [31]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

import matplotlib.pyplot as plt
import mlxtend.frequent_patterns
import mlxtend.preprocessing
import numpy

## Segmentación de clientes

## Canasta de mercado

Se busca con esta herramientas encontrar aquellos productos que se venden juntos en los distintos locales

In [5]:
data = pd.read_csv("./dataset/coffee-shop-sales-revenue.csv", delimiter='|')
data.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


In [197]:
# Obtengo las ubicaciones de los locales
locales = data.drop_duplicates(subset=['store_location'])['store_location']
print(locales)

0      Lower Manhattan
17      Hell's Kitchen
105            Astoria
Name: store_location, dtype: object


In [160]:
# Extraigo las transacciones por locales
datos_por_local = []
for local in locales:
    ventas = data[data['store_location'] == local][['datetime', 'product_type']]
    datos_por_local.append(ventas)

In [178]:
# Extraigo los tipos de productos que se vendieron en el local
tmpVentas = datos_por_local[0]
productos = pd.DataFrame(datos_por_local[0]).drop_duplicates('product_type')['product_type'].tolist()
productos.insert(0, 'datetime')

In [179]:
# Convierto los productos en columnas
MBA_data = pd.DataFrame([productos])
MBA_data.columns = productos
MBA_data.head()

,datetime,Gourmet brewed coffee,Brewed Chai tea,Hot chocolate,Drip coffee,Scone,Barista Espresso,Brewed Black tea,Brewed Green tea,Biscotti,...,Premium Beans,Green beans,Espresso Beans,Housewares,Green tea,Black tea,Clothing,Organic Chocolate,Chai tea,House blend Beans
0,datetime,Gourmet brewed coffee,Brewed Chai tea,Hot chocolate,Drip coffee,Scone,Barista Espresso,Brewed Black tea,Brewed Green tea,Biscotti,...,Premium Beans,Green beans,Espresso Beans,Housewares,Green tea,Black tea,Clothing,Organic Chocolate,Chai tea,House blend Beans


In [180]:
# NO EJECUTAR A MENOS QUE SEA NECESARIO
# Construyo la matriz en base a productos comprados por fecha
tmpVentas.head()

for index, row in tmpVentas.iterrows():
    fila = []
    fila.append(row['datetime'])
    #print(row['product_type'])
    for col in MBA_data.iloc[:, 1:]:
        if row['product_type'] == col:
            fila.append(1)
        else:
            fila.append(0)
    MBA_data.loc[len(MBA_data)] = fila
    
MBA_data = MBA_data.iloc[1:].reset_index(drop=True)
print(len(tmpVentas))
print(len(MBA_data))

47782
47782


In [193]:
# Agrupamos las columnas que tienen la misma fecha
MBA_data_grp = MBA_data.groupby('datetime').sum().reset_index()
MBA_data_grp = MBA_data_grp.drop(columns=['datetime'])
MBA_data_grp = MBA_data_grp.map(lambda x: True if x >= 1 else False)

In [196]:
# Realizamos el análisis
frequent_itemsets = apriori(MBA_data_grp, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift")
rules.sort_values(['support', 'confidence', 'lift'], axis = 0, ascending = False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
47,(Regular syrup),(Barista Espresso),0.058706,0.148631,0.058622,0.998567,6.718418,0.049897,593.971624,0.904240
46,(Barista Espresso),(Regular syrup),0.148631,0.058706,0.058622,0.394414,6.718418,0.049897,1.554352,0.999749
51,(Sugar free syrup),(Barista Espresso),0.023253,0.148631,0.023253,1.000000,6.728062,0.019796,inf,0.871637
50,(Barista Espresso),(Sugar free syrup),0.148631,0.023253,0.023253,0.156445,6.728062,0.019796,1.157894,1.000000
24,(Scone),(Barista Espresso),0.097330,0.148631,0.021570,0.221614,1.491032,0.007103,1.093761,0.364832
25,(Barista Espresso),(Scone),0.148631,0.097330,0.021570,0.145122,1.491032,0.007103,1.055905,0.386816
6,(Scone),(Brewed Chai tea),0.097330,0.142040,0.013267,0.136311,0.959670,-0.000558,0.993367,-0.044485
7,(Brewed Chai tea),(Scone),0.142040,0.097330,0.013267,0.093404,0.959670,-0.000558,0.995670,-0.046695
0,(Scone),(Gourmet brewed coffee),0.097330,0.146331,0.012285,0.126225,0.862597,-0.001957,0.976989,-0.149997
1,(Gourmet brewed coffee),(Scone),0.146331,0.097330,0.012285,0.083956,0.862597,-0.001957,0.985401,-0.157253


## Arbol de decisión